# M-PESA prediction
<img src="Charts/MPesaFinance.png" alt="Mpesa file" width="500" height="450">
<h2> Case study </h2>

<i><h4>Introduction</i></h4>

* M-Pesa is a mobile money service launched by Safaricom, Kenya’s leading telecommunications company, in 2007. It has revolutionized financial transactions, allowing users to send, receive, deposit, and withdraw money using their mobile phones. M-Pesa has played a significant role in financial inclusion, particularly for unbanked populations.

<i><h4>How M-Pesa Works</h4></i>

* M-Pesa enables users to perform transactions via USSD codes or the M-Pesa app. Users register with Safaricom and link their mobile   numbers to an M-Pesa account. Key services include:
- Depositing money at M-Pesa agent shops.
- Sending money to other users and non-users.
- Withdrawing cash from agents or ATMs.
- Paying bills (electricity, water, internet, school fees, etc.).
- Merchant payments through Lipa na M-Pesa.
- Accessing micro-loans and savings via M-Shwari and KCB M-Pesa.
- Overdraft services using Fuliza.<br>

<i><h4>Challenges and Risks</i></h4>

* High transaction costs: Some users find M-Pesa charges expensive for frequent transactions.
* less finance monitoring and alert on overspending money
* Alternative option on cheaper money spending areas

<i><h4>Solution</i></h4>

* Model a system using m-pesa stament to predict your spending
* provide an alternative of a cheaper spending activity
* provide an alert to user when the spending of money is high(email)

In [1]:
import pandas as pd
import numpy as np
import PyPDF2
import pikepdf
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from tabula.io import read_pdf 
from dateutil import parser 
from sqlalchemy import create_engine
import mysql.connector

In [2]:
# decrepting the file pdf
input_path = "/Users/briankimanzi/Downloads/Mpesa pdfs/Statement_All_Transactions_20240901_20250301.pdf"
output_path = "/Users/briankimanzi/Downloads/Mpesa pdfs/Tracking.pdf"
password = "102030"

with pikepdf.open(input_path, password=password) as pdf:
    pdf.save(output_path)
path = output_path

In [3]:
# getting number of pages on the pdf
with open(path, 'rb') as file:
    pdf_reader = PyPDF2.PdfReader(file)
    num_pages = len(pdf_reader.pages)

In [5]:
# extracting data from mpesa pages
def get_data():
    for page_number in range(1, num_pages+1):
        if page_number == 1:
            df =read_pdf(path, pages=page_number)
            data=df[1]
            return_df =data
        else:
            df = read_pdf(path, pages=page_number)
            data=df[0]

        if page_number != 1:
            return_df = pd.concat([return_df, data])

    return return_df.reset_index(drop=True)
working_data = get_data()

In [6]:
working_data

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance,Unnamed: 0
0,TC17XS3U67,2025-03-01 15:55:18,Customer Transfer to 254718***450 -\rGideon Ki...,COMPLETED,0.00,80.00,27.00,NaN
1,TC14XHM64C,2025-03-01 14:49:55,M-Shwari Deposit,COMPLETED,0.00,"1,000.00",107.00,NaN
2,TC11XF54B3,2025-03-01 14:34:40,Funds received from 0706***880 - sheila\rmercy...,COMPLETED,"1,100.00",0.00,"1,107.00",NaN
3,TC13WSE6N5,2025-03-01 12:11:53,Pay Bill Online to 4076749 - TINGG Acc.\r25410...,COMPLETED,0.00,10.00,7.00,NaN
4,TBS1THWSOL,2025-02-28 17:50:41,Customer Transfer to 254716***061 -\rNAOMI NYA...,COMPLETED,0.00,10.00,17.00,NaN
...,...,...,...,...,...,...,...,...
269,TAB3U80KYL,2025-01-11 20:34:49,M-Shwari Withdraw,COMPLETED,50.0,0.0,79.0,NaN
270,TAA7PUAYUX,2025-01-10 20:19:56,Pay Bill Charge,COMPLETED,0.0,2.0,29.0,NaN
271,TAA7PUAYUX,2025-01-10 20:19:56,Pay Bill Online to 4093275 - Direct Pay\rLimit...,COMPLETED,0.0,20.0,31.0,NaN
272,TAA4PU1I32,2025-01-10 20:18:40,M-Shwari Withdraw,COMPLETED,50.0,0.0,51.0,NaN


In [7]:
# Reading other dataset
Data1 = pd.read_csv('/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/katra.csv')
Data2 = pd.read_csv('/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/Nashipae.csv')

In [8]:
Data1

,Receipt,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
0,TD71R8CU5T,68,2025,4,7,0,12,24,25,AIRTIME PURCHASE,AIRTIME,14.0,WITHDRAW,0.71
1,TD48BNDSIK,67,2025,4,4,4,5,10,29,PAY BILL,SAFARICOM HOME,3699.0,WITHDRAW,96822.71
2,TCU0NDJPKK,66,2025,3,30,6,7,13,23,PAY BILL CHARGES,TRANSACTION COST,10.0,WITHDRAW,27.71
3,TCU0NDJPKK,66,2025,3,30,6,7,13,23,PAY BILL,KPLC PREPAID,1000.0,WITHDRAW,37.71
4,TCS0HGRZ5U,65,2025,3,28,4,20,9,27,AIRTIME PURCHASE,AIRTIME,5.0,WITHDRAW,1037.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,SJC9SXS74T,2,2024,10,12,5,11,49,43,PAY BILL CHARGES,TRANSACTION COST,5.0,WITHDRAW,126.71
131,SJC9SXS74T,2,2024,10,12,5,11,49,43,PAY BILL,KPLC PREPAID,500.0,WITHDRAW,131.71
132,SJB8QWVJN2,1,2024,10,11,4,19,36,51,RECEIVED FUNDS,SAFIA SOYAN,500.0,PAID IN,631.71
133,SJB2QIB142,1,2024,10,11,4,18,34,48,BUSINESS PAYMENT,CO-OP BANK VIA API,120.0,PAID IN,131.71


In [9]:
Data2

,Receipt,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
0,TD349XUJFS,154,2025,4,3,3,18,27,59,BUY GOODS,SAWA SHOP 4,65.0,WITHDRAW,1260.75
1,TD399XA2KP,154,2025,4,3,3,18,26,0,BUY GOODS,MOHAMED DULACHA ALI,120.0,WITHDRAW,1325.75
2,TD359WGPPX,154,2025,4,3,3,18,22,0,BUY GOODS,JOANINA KARIMI RUKARIA 6,70.0,WITHDRAW,1445.75
3,TD359Q25IZ,154,2025,4,3,3,17,53,0,PAY BILL CHARGES,TRANSACTION COST,57.0,WITHDRAW,1515.75
4,TD359Q25IZ,154,2025,4,3,3,17,53,0,PAY BILL,CO- OPERATIVE BANK MONEY TRANSFER,10500.0,WITHDRAW,1572.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,SJ49WDO7A3,1,2024,10,4,4,18,38,59,BUY GOODS,JACKSON MWAURA MUGURO,50.0,WITHDRAW,2330.81
1074,SJ49WC912X,1,2024,10,4,4,18,31,59,BUY GOODS,BK 911 CHICKEN,170.0,WITHDRAW,2380.81
1075,SJ41W9ZSH5,1,2024,10,4,4,18,22,0,BUY GOODS,JUDITH KATHURE MWITI,130.0,WITHDRAW,2550.81
1076,SJ40V94Q5K,1,2024,10,4,4,14,35,59,POCHI LA BIASHARA,MADERINA NYOROKA,50.0,WITHDRAW,2680.81


<h3><i>Data cleaning </i></h3>

In [10]:
# function to remove the commas on the dataset
def remove_comma(x):
    """This function removes commans on the dataset
    parameters: x
    return x
    """
    x = str(x)
    x = x.replace(',', '')
    return x

In [11]:
data = working_data.copy()
data['Paid in'] = data['Paid in'].apply(lambda x: remove_comma(x))
data['Withdraw\rn'] = data['Withdraw\rn'].apply(lambda x : remove_comma(x))
data['Balance'] = data['Balance'].apply(lambda x: remove_comma(x))

In [12]:
# changing data type
data = data.astype({
    'Paid in': float,
    'Withdraw\rn':float,
    'Balance':float
})

In [13]:
# dropping unwanted columns
data.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
# Anonymising the dataset 
def transaction(x):
    x = str(x).strip()
    if x.startswith('Merchant Payment'):
        index = x.find(' - ')+2
        name = x[index:].strip().upper()
        
        return 'BUY GOODS', name

    elif x.startswith('Deposit of funds'):
        index = x.find(' - ')
        name = x[index:].strip().upper()
        
        return 'AGENT DEPOSIT', name

    elif x.startswith('OD Loan Repayment'):
        return 'FULIZA REPAYMENT', 'FULIZA'

    elif x.startswith('OverDraft of Credit Party'):
        return 'FULIZA TAKEN', 'FULiZA'

    elif x.startswith('M-Shwari Deposit'):
        return 'M-SHWARI DEPOSIT FROM M-PESA', 'M-SHWARI'
        
    elif x.startswith('KCB M-PESA Deposit'):
        return 'KCB M-PESA DEPOSIT FROM M-PESA', 'KCB DEPOSIT'

    elif x.startswith('KCB M-PESA Withdraw'):
        return 'KCB M-PESA WITHDRAW FROM M-PESA', 'KCB WITHDRAW'

    elif x.startswith('Customer Transfer'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
                
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        return 'SEND MONEY', name

    elif x.startswith('M-Shwari Withdraw'):
        
        return 'M-SHWARI WITHDRAW FROM M-PESA', 'M-SHWARI WITHDRAW'

    elif x.startswith('Pay Bill'):
        if x.strip() == 'Pay Bill Charge':
            
            return 'PAY BILL CHARGES', 'TRANSACTION COST'
        else:
            index = x.find(' - ')+2
            end = x.lower().find('acc')
            name = x[index:end].strip().upper()
            
            return 'PAY BILL', name

    elif x.startswith('Funds received'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        
        return 'RECEIVED FUNDS', name

    elif x.startswith('Customer Payment to Small Business') or x.startswith('Customer Send Money'):
        index = x.find(' - ')
        to_search = x[index:]
        last = 1

        for i in range(len(to_search)):
            try:
                int(to_search[i])
                last = i
            except:
                v = 10

        name = x[index+last+2:].strip().upper()
        
        return 'POCHI LA BIASHARA', name

    elif x.startswith('Airtime Purchase'):
        return 'AIRTIME PURCHASE', 'AIRTIME'

    elif x.startswith('Business Payment From'):
        index = x.find(' - ')
        end = x.lower().find('via')
        to_search = x[index:]
        last = 1
        name = x[index:end].strip().upper()
        return 'FUNDS RECEIVED FROM BUSINESS', name

    elif x.startswith('Customer Transfer of Funds Charge'):
        return 'TRANSACTION COST', 'TRANSACTION COST'
    
    elif x.startswith('Buy Bundles Online'):
        return 'BUNDLES PURCHASE', 'BUNDES PURCHASE'
    
    elif x.startswith('Customer Withdrawal'):
        index = x.find(' - ')
        name = x[index:].strip().upper()
        return 'CASH WITHDRAWAL', name
    
    elif x.startswith('Withdrawal Charge'):
        return 'CASH WITHDRAWAL CHARGES', "TRANSACTION COST"
    
    elif x.startswith('Savings Contribution'):
        return 'TO HUSTLER FUND SAVINGS', 'HUSTLER FUND'
    
    elif x.startswith('Term Loan Disbursement for H- Fund') or x.startswith('Term Loan Disbursement for H-Fund'):
        return 'HUSTLER FUND Disbursement'.upper(), 'HUSTLER FUND'
    
    elif x.startswith('Term Loan Repayment for H- Fund') or x.startswith('Term Loan Repayment for H-Fund'):
        return 'HUSTLER FUND REPAYMENT', 'HUSTLER FUND'
    
    else:
        return 'UNIDENTIFIED', 'UNIDENTIFIED'

In [15]:
data['Details'] = data['Details'].apply(lambda x: x.replace('\r', ' '))

In [16]:
details = list(data['Details'].apply(lambda x: transaction(x)).values)
transactionType = [i[0] for i in details]
TransactionParty = [i[1] for i in details]

In [17]:
working_data.drop("Unnamed: 0", axis=1, inplace=True)

In [20]:
# changing the date format
def change_date(date):
    date = parser.parse(date)
    return (date.year, date.month, date.day, date.weekday(),date.hour, date.minute, date.second)

In [21]:
date = data['Completion Time'].apply(lambda x: change_date(x)).values
Year = [i[0] for i in date]
Month = [i[1] for i in date]
Date = [i[2] for i in date]
Weekday = [i[3] for i in date]
Hour = [i[4] for i in date]
Minute = [i[5] for i in date]
Seconds = [i[6] for i in date]

transactionDay = []

reverseDay = Date.copy()[::-1]
datey = reverseDay[0]

d = 1

for i in reverseDay:
    if i == datey:
        transactionDay.append(d)
    else:
        datey = i
        d += 1
        transactionDay.append(d)

transactionDay.reverse()

In [22]:
receipt = list(data['Receipt No'].values)

In [23]:
paid_in = data['Paid in'].fillna('NAN_VALUE').values

In [24]:
def withdrawAmount(x):
    try:
        if x < 0:
            return -x
        else:
            return x
    except:
        return x

In [25]:
withdraw = list(data['Withdraw\rn'].apply(lambda x: withdrawAmount(x)).values)
withdraw                

[80.0,
 1000.0,
 0.0,
 10.0,
 10.0,
 15.0,
 10.0,
 0.0,
 55.0,
 50.0,
 13.0,
 1000.0,
 100.0,
 10.0,
 15.0,
 100.0,
 55.0,
 60.0,
 20.0,
 70.0,
 50.0,
 20.0,
 15.0,
 20.0,
 120.0,
 10.0,
 0.0,
 10.0,
 10.0,
 240.0,
 10.0,
 0.0,
 35.0,
 10.0,
 20.0,
 10.0,
 10.0,
 50.0,
 7.0,
 135.0,
 20.0,
 0.0,
 30.0,
 0.0,
 10.0,
 0.0,
 50.0,
 0.0,
 25.0,
 0.0,
 40.0,
 0.0,
 30.0,
 0.0,
 10.0,
 0.0,
 2000.0,
 0.0,
 20.0,
 20.0,
 50.0,
 100.0,
 10.0,
 0.0,
 50.0,
 0.0,
 50.0,
 0.0,
 50.0,
 0.0,
 50.0,
 10.0,
 75.0,
 0.0,
 10.0,
 2.0,
 5.0,
 0.0,
 2.0,
 40.0,
 80.0,
 100.0,
 0.0,
 20.0,
 0.0,
 100.0,
 100.0,
 0.0,
 30.0,
 145.0,
 0.0,
 250.0,
 0.0,
 0.0,
 60.0,
 80.0,
 10.0,
 7.0,
 130.0,
 0.0,
 100.0,
 7.0,
 300.0,
 0.0,
 3.0,
 50.0,
 0.0,
 70.0,
 10.0,
 0.0,
 10.0,
 200.0,
 0.0,
 50.0,
 7.0,
 200.0,
 0.0,
 4800.0,
 0.0,
 10.0,
 70.0,
 60.0,
 0.0,
 100.0,
 15.0,
 10.0,
 20.0,
 10.0,
 10.0,
 7.0,
 150.0,
 20.0,
 0.0,
 15.0,
 15.0,
 15.0,
 20.0,
 10.0,
 10.0,
 10.0,
 15.0,
 15.0,
 20.0,
 50.0,
 10.0,
 2

In [26]:
transaction_Data = []

for i in range(len(paid_in)):
    try:
        x = float(paid_in[i])
        transaction_Data.append((x, 'PAID IN'))
    except:
        x = float(withdraw[i])
        transaction_Data.append((x, 'WITHDRAW'))

In [27]:
transaction_amount = [i[0] for i in transaction_Data]
paid_in_or_withdraw = [i[1] for i in transaction_Data]

In [28]:
balance =list(data["Balance"].values)

In [29]:
final_data = pd.DataFrame({
    "Receipt":receipt,
    "transaction_Day": transactionDay,
    "Year" : Year,
    "Month" : Month,
    "Date" : Date,
    "Weekday" : Weekday,
    "Hour" : Hour,
    "Minute" : Minute,
    "Seconds" : Seconds,
    "Transaction_type" : transactionType,
    "Transaction_party" : TransactionParty,
    "Transaction_amount" : transaction_amount,
    "paid_in_or_Withdraw" : paid_in_or_withdraw,
    "Balance" : balance
})
final_data.set_index('Receipt', inplace=True)
final_data

,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
Receipt,,,,,,,,,,,,,
TC17XS3U67,49,2025,3,1,5,15,55,18,SEND MONEY,GIDEON KIPAMET KAIYIAN,0.0,PAID IN,27.0
TC14XHM64C,49,2025,3,1,5,14,49,55,M-SHWARI DEPOSIT FROM M-PESA,M-SHWARI,0.0,PAID IN,107.0
TC11XF54B3,49,2025,3,1,5,14,34,40,RECEIVED FUNDS,SHEILA MERCY NDUSYA,1100.0,PAID IN,1107.0
TC13WSE6N5,49,2025,3,1,5,12,11,53,PAY BILL,TINGG,0.0,PAID IN,7.0
TBS1THWSOL,48,2025,2,28,4,17,50,41,SEND MONEY,NAOMI NYAMBURA MBURU,0.0,PAID IN,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TAB3U80KYL,2,2025,1,11,5,20,34,49,M-SHWARI WITHDRAW FROM M-PESA,M-SHWARI WITHDRAW,50.0,PAID IN,79.0
TAA7PUAYUX,1,2025,1,10,4,20,19,56,PAY BILL CHARGES,TRANSACTION COST,0.0,PAID IN,29.0
TAA7PUAYUX,1,2025,1,10,4,20,19,56,PAY BILL,DIRECT PAY LIMITED 1,0.0,PAID IN,31.0


In [30]:
# joining all the datasets together
Data1.reset_index(inplace=True)
Data2.reset_index(inplace=True)
final_data.reset_index(inplace=True)

In [31]:
combined_data = pd.concat([final_data, Data1, Data2], ignore_index=True)

In [32]:
combined_data.set_index('Receipt', inplace=True)

In [33]:
combined_data.drop(columns='index', errors='coerce', inplace=True)

In [34]:
combined_data

,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
Receipt,,,,,,,,,,,,,
TC17XS3U67,49,2025,3,1,5,15,55,18,SEND MONEY,GIDEON KIPAMET KAIYIAN,0.0,PAID IN,27.00
TC14XHM64C,49,2025,3,1,5,14,49,55,M-SHWARI DEPOSIT FROM M-PESA,M-SHWARI,0.0,PAID IN,107.00
TC11XF54B3,49,2025,3,1,5,14,34,40,RECEIVED FUNDS,SHEILA MERCY NDUSYA,1100.0,PAID IN,1107.00
TC13WSE6N5,49,2025,3,1,5,12,11,53,PAY BILL,TINGG,0.0,PAID IN,7.00
TBS1THWSOL,48,2025,2,28,4,17,50,41,SEND MONEY,NAOMI NYAMBURA MBURU,0.0,PAID IN,17.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SJ49WDO7A3,1,2024,10,4,4,18,38,59,BUY GOODS,JACKSON MWAURA MUGURO,50.0,WITHDRAW,2330.81
SJ49WC912X,1,2024,10,4,4,18,31,59,BUY GOODS,BK 911 CHICKEN,170.0,WITHDRAW,2380.81
SJ41W9ZSH5,1,2024,10,4,4,18,22,0,BUY GOODS,JUDITH KATHURE MWITI,130.0,WITHDRAW,2550.81


In [35]:
# connecting to the database
DB_USER = "root"
DB_PASSWORD = "673gezmigel"
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "mpesa"

# creating engine
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
# test connection

try:
    with engine.connect() as conn:
        print("Connection to database is 👍🏼")
        
    combined_data.reset_index(inplace = True)
    combined_data.to_sql(name='Mpesa_Statement', con=engine, if_exists='replace', index=False)
    
except Exception as e:
    print(f"connection Failed {e} 👎🏼")

Connection to database is 👍🏼


,Receipt,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
0,TD71R8CU5T,68,2025,4,7,0,12,24,25,AIRTIME PURCHASE,AIRTIME,14.0,WITHDRAW,0.71
1,TD48BNDSIK,67,2025,4,4,4,5,10,29,PAY BILL,SAFARICOM HOME,3699.0,WITHDRAW,96822.71
2,TCU0NDJPKK,66,2025,3,30,6,7,13,23,PAY BILL CHARGES,TRANSACTION COST,10.0,WITHDRAW,27.71
3,TCU0NDJPKK,66,2025,3,30,6,7,13,23,PAY BILL,KPLC PREPAID,1000.0,WITHDRAW,37.71
4,TCS0HGRZ5U,65,2025,3,28,4,20,9,27,AIRTIME PURCHASE,AIRTIME,5.0,WITHDRAW,1037.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,SJC9SXS74T,2,2024,10,12,5,11,49,43,PAY BILL CHARGES,TRANSACTION COST,5.0,WITHDRAW,126.71
131,SJC9SXS74T,2,2024,10,12,5,11,49,43,PAY BILL,KPLC PREPAID,500.0,WITHDRAW,131.71
132,SJB8QWVJN2,1,2024,10,11,4,19,36,51,RECEIVED FUNDS,SAFIA SOYAN,500.0,PAID IN,631.71
133,SJB2QIB142,1,2024,10,11,4,18,34,48,BUSINESS PAYMENT,CO-OP BANK VIA API,120.0,PAID IN,131.71


# <i>Extrapolatory Data Analysis</i>
* <h3>Finding insight and correlation on dataset by answering the following question</h3>

- What day of the week do I spend the most money?  
- How does spending vary across different months or years?
- At what time of day do most transactions happen?
- Is there a particular week or date where spending spikes?

<h3> Spending Patterns</h3>

- Which transaction types (e.g., Buy Goods, PayBill, Withdraw) do I use most?
- What’s the average transaction amount by type?
- Are there more cash-ins (Received) or cash-outs (Withdrawals)?

<h3>  People or Places You Transact With</h3>

- Which parties do I send/receive money to/from the most?
- Who do I spend the most money on?

<h3> Balance & Financial </h3>

- What’s my balance trend over time?
- Do larger transactions lead to lower account balances?



In [36]:
Rows, Columns = combined_data.shape
print(f"Rows: {Rows} Columns: {Columns}")

Rows: 1487 Columns: 14


In [ ]:
# What day of the week do I spend the most money?
# mpesa color code #439534
plt.figure(figsize=(18, 7))
plt.style.use('seaborn-v0_8-darkgrid')
plot = sns.barplot(data=combined_data,
            x='Weekday',  y='Transaction_amount', color='#439534', errorbar=None,
           )
plt.title('Transaction spend during weekday', fontsize=15)
plt.xlabel('Weekday', fontsize=15)
plt.xticks(fontsize=15)
plt.ylabel('Transaction Amount', fontsize=15)
plt.yticks(fontsize=15)
weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
plot.set_xticklabels([weekday_names[i] for i in plot.get_xticks()])
plt.savefig("ModelCharts/Transaction_spend_during_weekday.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# How does spending vary across different months?
plt.figure(figsize=(18, 7))
plot = sns.lineplot(data=combined_data, x='Month', y='Transaction_amount', color='#439534', errorbar=None)
months_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
existing_months = sorted(combined_data['Month'].unique())
plot.set_xticks(existing_months)
plot.set_xticklabels([months_names[int(i)-1] for i in existing_months])
plot.set_xlim(1, 12)
plt.title("Monthly Transaction over the Months", fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel("Month", fontsize=15)
plt.ylabel("Transaction Amount", fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Monthly_transaction_over_the_months.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# At what time of day do most transactions happen?
fig, ax1 = plt.subplots(figsize=(18, 7))
sns.histplot(data=combined_data, x='Hour', color='#439534', binwidth=1, alpha=0.7, ax=ax1)
ax1.set_ylabel('Number of transaction in Hour', color='#439534')
ax1.tick_params(axis='y', labelcolor='#439534')
ax1.set_xlim(0, 23)
ax2 = ax1.twinx()
ax2 = sns.lineplot(data=combined_data.groupby('Hour')['Transaction_amount'].sum().reset_index(),x='Hour',y='Transaction_amount',
                   color='red',linewidth=2.5,ax=ax2
                  )
ax2.set_ylabel('Total Transaction Amount', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Transaction Frequency and Amount by Hour of Day', fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel("Hour", fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Transaction_frequency_and_amount_by_hour_of_day.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Is there a particular date where spending spikes?
plt.figure(figsize=(18, 7))
plot = sns.lineplot(data=combined_data, x='Date', y='Transaction_amount', errorbar=None, color='#439534')
plot.set_xlim(1, 32)
plt.title("Daily Transaction where spending spikes", fontsize=15)
plt.xticks(fontsize=15)
plt.xlabel("Date", fontsize=15)
plt.ylabel("Transaction Amount", fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Daily_transction_where_spending_spikes.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Which transaction types do I use most?
plt.figure(figsize=(18, 7))
sort = combined_data.sort_values(by='Transaction_type',ascending=False)
sns.countplot(data=combined_data, y='Transaction_type', color='#439534',
             order=combined_data['Transaction_type'].value_counts().index)
plt.title('Most used Transaction Type', fontsize=20)
plt.xlabel('Count', fontsize=18)
plt.xticks(fontsize=15)
plt.ylabel('Transaction Type', fontsize=18)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Most_used_transaction_type.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# What’s the average transaction amount by type?
Grouped_avg_type = (combined_data.groupby('Transaction_type')['Transaction_amount']
                    .mean().sort_values(ascending=False).reset_index())

plt.figure(figsize=(18, 7))
sns.barplot(data=Grouped_avg_type, y='Transaction_type', x='Transaction_amount', color='#439534')
plt.title('Average Transaction amount by Transaction_type', fontsize=20)
plt.xlabel('Transaction_Amount', fontsize=18)
plt.ylabel('Transaction_Type', fontsize=18)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Average_transaction_amount_by_transaction_type.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Are there more cash-ins (Paid In) or cash-outs (Withdrawals)?
paid_in = combined_data['paid_in_or_Withdraw'].value_counts()
plt.figure(figsize=(15, 7))
fig = go.Figure(go.Pie(labels=paid_in.index, values=paid_in.values, hole=.5, 
                      marker_colors = ['#439534', 'red'],
                      textinfo='percent+label',
                      hoverinfo='value'))
fig.update_layout(
    title_text='(Paid In) vs. (Withdraws)',
    title_x=0.5,  # Center title
    showlegend=False,  
    height=500,
)
fig.show()
fig.write_image("ModelCharts/(Paid_in)vs(withdraws).png", scale=3)

In [ ]:
# Which parties do I send/receive money to/from the most?
plt.figure(figsize=(18,7))
top_Parties = (combined_data.groupby('Transaction_party')['Transaction_amount']
               .sum().sort_values(ascending=False).head(20).reset_index())
sns.barplot(data=top_Parties,
           x='Transaction_amount',
           y='Transaction_party',
           color='#439534')
plt.title('Top 20 Most spend Parties (Send/Receive) Money', fontsize=20)
plt.xlabel('Transaction Amount', fontsize=18)
plt.ylabel('Transaction Parties', fontsize=18)
plt.xticks(fontsize=18)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Top20_most_spend_parties(send_receive)_money.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Who do I spend the most money on
plt.figure(figsize=(18,7))
filtered = combined_data[combined_data["paid_in_or_Withdraw"] == "WITHDRAW"]
spend_most_money = (filtered.groupby("Transaction_party")["Transaction_amount"]
                    .sum().head(20).sort_values(ascending=False).reset_index())
sns.barplot(data=spend_most_money, x="Transaction_amount", y="Transaction_party", color="#439534")
plt.title("Top 20 spending under withdraw", fontsize=20)
plt.xlabel("Transaction_Amount", fontsize=18)
plt.ylabel("Transaction_Party", fontsize=18)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/top20_spending_under_withdraw.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# What’s my balance trend over time?
plt.figure(figsize=(18,7))
sns.lineplot(data=combined_data, x='Date', y='Balance', color="#439534", errorbar=None)
plt.title("My Balance Trend Over Time", fontsize=17)
plt.xlabel("Date", fontsize=17)
plt.ylabel("Balance", fontsize=17)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlim(1,32)
plt.savefig("ModelCharts/My_balance_trend_over_time.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# Do larger transactions lead to lower account balances?
plt.figure(figsize=(18,7))
Low_balance = 100
big_transaction = combined_data['Transaction_amount'].quantile(.75)

sns.scatterplot(
    data=combined_data, x='Transaction_amount', y='Balance',
    hue = combined_data['Transaction_amount'] < Low_balance,
    palette={True: 'red', False: "#439534"},
    style=combined_data["Transaction_amount"] > big_transaction,
    markers={True: 'X', False: 'o'},
    size=combined_data['Transaction_amount'],
    sizes=(20, 200),
    alpha=0.9
    )
# Add reference lines
plt.axvline(big_transaction, color='blue', linestyle='--', label='Big Transaction Threshold')
plt.axhline(Low_balance, color='red', linestyle='--', label='Low Balance Threshold')

plt.title('Transaction Impact on Account Balance', fontsize=20)
plt.xlabel('Transaction Amount', fontsize=15)
plt.ylabel('Account Balance', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend(title='Risk Indicators', bbox_to_anchor=(1.05, 1), loc='upper left')

#  risk zones
plt.text(
    x=big_transaction*1.1, 
    y=Low_balance*0.5, 
    s='High Risk Zone', 
    color='red',
    fontsize=12,
    bbox=dict(facecolor='white', alpha=0.8)
)

plt.tight_layout()
plt.savefig("ModelCharts/Transaction_impact_on_account_balance.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance,starting_balance,balance_after
0,49,2025,3,1,5,15,55,18,SEND MONEY,GIDEON KIPAMET KAIYIAN,0.0,PAID IN,27.00,5000,5000.0
1,49,2025,3,1,5,14,49,55,M-SHWARI DEPOSIT FROM M-PESA,M-SHWARI,0.0,PAID IN,107.00,5000,5000.0
2,49,2025,3,1,5,14,34,40,RECEIVED FUNDS,SHEILA MERCY NDUSYA,1100.0,PAID IN,1107.00,5000,3900.0
3,49,2025,3,1,5,12,11,53,PAY BILL,TINGG,0.0,PAID IN,7.00,5000,5000.0
4,48,2025,2,28,4,17,50,41,SEND MONEY,NAOMI NYAMBURA MBURU,0.0,PAID IN,17.00,5000,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,1,2024,10,4,4,18,38,59,BUY GOODS,JACKSON MWAURA MUGURO,50.0,WITHDRAW,2330.81,5000,4950.0
1483,1,2024,10,4,4,18,31,59,BUY GOODS,BK 911 CHICKEN,170.0,WITHDRAW,2380.81,5000,4830.0
1484,1,2024,10,4,4,18,22,0,BUY GOODS,JUDITH KATHURE MWITI,130.0,WITHDRAW,2550.81,5000,4870.0
1485,1,2024,10,4,4,14,35,59,POCHI LA BIASHARA,MADERINA NYOROKA,50.0,WITHDRAW,2680.81,5000,4950.0


<h2> Machine learning prediction </h2>

- <h3> <i>Finding insight and Predicting outcome of certain events </i> </h3>

In [48]:
# finding the correlation on the floating/numerical value
corr = combined_data.select_dtypes(include=float)
corr.reset_index(inplace=True)
corr.drop(columns='index', errors='coerce', inplace=True)

In [ ]:
# corr = corr.corr()
plt.figure(figsize=(10,7))
sns.heatmap(data=corr, annot=True, cmap=['#E53631', '#439534'])
plt.title("Heatmap correlationon between Transaction_amount and Balance", fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.savefig("ModelCharts/Heatmap_correlation_btwn_transaction_amount_and_balance.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [50]:
prediction_data = combined_data.copy()

In [51]:
# changing certain columns into categories
category_columns = ['Transaction_type', 'Transaction_party', 'paid_in_or_Withdraw']
for columns in category_columns:
    prediction_data[columns] = prediction_data[columns].astype('category')

In [ ]:
boxcolumns = prediction_data.select_dtypes(include=['int', 'float'])
plt.figure(figsize=(18, 7))
sns.boxplot(data=boxcolumns, patch_artist=True, boxprops=dict(facecolor='lightblue', color='blue'),
             medianprops=dict(color='red'), whiskerprops=dict(color='blue'), capprops=dict(color='blue'), 
              flierprops=dict(marker='o', color='grey', alpha=0.5))
plt.title("Checking outliners on the dataset using the numerical and float values", fontsize=15)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.savefig("ModelCharts/Checking_outliners_on_the_dataset.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [53]:
# function for checking missing values
def missing_values(df: pd.DataFrame):
    missing_percentage = (df.isna().sum() / len(df)) * 100
    missing_df = pd.DataFrame({'Columns': df.columns, "Missing %": missing_percentage})
    missing_df = missing_df[missing_df['Missing %'] > 0].sort_values(by='Missing %', ascending=False)

    return missing_df

In [54]:
# returns no missing values on the prediction_data set
missing_values(prediction_data)

,Columns,Missing %


In [55]:
# checking outliners on the dataset
def check_outliners(df):
    """This function uses the quantile calculation to check outliners on the data
    Parameters used are df for the dataset and the function 
    1.5 threshold is somewhat arbitrary but works well 
    in practice to flag unusual points without being too sensitive.
    returns outliner dataframe"""
    outliners_data = []
    for columns in df.select_dtypes(include=['int', 'float']).columns:
        quant1 = df[columns].quantile(0.25)
        quant3 = df[columns].quantile(0.75)
        IQR = quant3 - quant1

        # compute outliner boundaries
        lowerBoundary = quant1 - 1.5 * IQR
        upperBoundary = quant3 + 1.5 * IQR

        # count the outliners on the dataset
        outliners = df[(df[columns] < lowerBoundary) | (df[columns] > upperBoundary)]
        outliner_count = outliners.shape[0]
        outliner_per = (outliner_count / df.shape[0]) * 100

        # append data to the list
        outliners_data.append({
            'columns':columns,
            'outliner_count': outliner_count,
            'outliner_percentage' : outliner_per,
            'lower Boundary': lowerBoundary,
            'upper Boundary': upperBoundary
        })
    outlinerDf = pd.DataFrame(outliners_data)
    return outlinerDf 

In [56]:
check_outliners(prediction_data)

,columns,outliner_count,outliner_percentage,lower Boundary,upper Boundary
0,transaction_Day,0,0.000000,-76.500,199.500
1,Year,0,0.000000,2022.500,2026.500
2,Month,0,0.000000,-11.500,24.500
3,Date,0,0.000000,-10.500,41.500
4,Weekday,0,0.000000,-5.000,11.000
5,Hour,0,0.000000,-2.000,30.000
6,Minute,0,0.000000,-32.500,83.500
7,Seconds,0,0.000000,-88.500,147.500
8,Transaction_amount,196,13.180901,-139.690,286.150
9,Balance,183,12.306658,-2950.405,5050.675


In [57]:
# dropping outliners
def drop_outliners(df: pd.DataFrame):
    no_outliners = df.copy()
    for columns in df.select_dtypes(include=['int', 'float']).columns:
        q1 = df[columns].quantile(.25)
        q3 = df[columns].quantile(.75)
        IQR = q3 - q1

        lowerbound = q1 - 1.5 * IQR
        upperbound = q3 + 1.5 * IQR

        # replace outliers on the data set
        no_outliners[columns] = df[columns].apply(lambda x : np.nan if (pd.notna(x) and (x < lowerbound or x > upperbound)) else x)
    return no_outliners
        

In [58]:
clean_data = drop_outliners(prediction_data)

In [59]:
missing_values(clean_data)

,Columns,Missing %
Transaction_amount,Transaction_amount,13.180901
Balance,Balance,12.306658


In [60]:
# filling missing values with median value
def fill_na(df):
    return df.fillna(df.median())

In [61]:
clean_data[['Balance', 'Transaction_amount']] = fill_na(clean_data[['Balance', 'Transaction_amount']])

In [62]:
missing_values(clean_data)

,Columns,Missing %


In [63]:
from sklearn.preprocessing import LabelEncoder

def custom_encoder(df: pd.DataFrame, encoding_type: str = 'label', 
                   special_column: str = 'Transaction_party', threshold: int = 10) -> pd.DataFrame:
    df_encoded = df.copy()

    # Select all object or category columns
    category_columns = df_encoded.select_dtypes(include='category').columns

    for col in category_columns:
        # Special case: Transaction_party → encode only frequent senders
        if col == special_column:
            sender_counts = df_encoded[col].value_counts()
            frequent_senders = sender_counts[sender_counts > threshold].index
            df_encoded[col] = df_encoded[col].apply(lambda x: x if x in frequent_senders else 'Other')
            
            label_enc = LabelEncoder()
            df_encoded[col] = label_enc.fit_transform(df_encoded[col].astype(str))
        
        else:
            # Normal encoding
            if encoding_type == 'label':
                label_enc = LabelEncoder()
                df_encoded[col] = label_enc.fit_transform(df_encoded[col].astype(str))
            
            elif encoding_type == 'onehot':
                df_encoded = pd.get_dummies(df_encoded, columns=[col], drop_first=True)
            
            else:
                print(f'Unsupported encoding type for column {col}. Use "label" or "onehot".')
    
    return df_encoded

In [109]:
df_encoding = custom_encoder(clean_data, encoding_type='label', special_column='Transaction_party', threshold=15)
df_encoding.sample(20)

,Receipt,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
343,TA514QQT7T,31,2025,1,5,6,22,34,40,2,0,20.0,1,1395.71
17,TBO8B37JCG,45,2025,2,24,0,17,50,21,4,13,0.0,0,1400.00
1473,SJ541N67DE,2,2024,10,5,5,18,11,0,15,13,45.0,1,1140.81
1386,SJH9EGDPA1,14,2024,10,17,3,13,27,0,4,15,45.0,1,3318.22
268,TAB5U87H71,2,2025,1,11,5,20,35,45,19,13,0.0,0,9.00
308,TBL1WOU5RD,50,2025,2,21,4,17,9,2,18,13,60.0,0,220.50
130,TB99F3LZRD,30,2025,2,9,6,20,21,50,19,13,0.0,0,286.00
427,TD22320TXU,153,2025,4,2,2,11,15,0,4,1,70.0,1,220.50
1056,SL52ACD75A,54,2024,12,5,3,9,44,0,7,14,10.0,1,220.50
1071,SL3032AGR2,52,2024,12,3,1,16,50,0,13,13,60.0,1,220.50


In [110]:
# splitting the dataset into training and testing
x = df_encoded[['Transaction_amount']]
y = df_encoded['Transaction_amount']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [111]:
# scaling the values
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def get_scaler(scaling_type: str = 'standard'):
    if scaling_type == 'standard':
        return StandardScaler()
    elif scaling_type == 'minmax':
        return MinMaxScaler()
    else:
        raise ValueError('Unsupported scaler, use standard or minmax')
def scale_train_test(x_train, x_test, scaling_type='standard'):
    scaler = get_scaler(scaling_type)
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    return x_train_scaled, x_test_scaled, scaler

In [71]:
x_train_scaled, x_test_scaled, scaler = scale_train_test(x_train, x_test, scaling_type='standard' )

In [113]:
# saving the model to file 
import pickle

# with open('ModelsPrediction/scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

with open('ModelsPrediction/df_encoding.pkl', 'wb') as f:
    pickle.dump(df_encoding, f)

In [74]:
from sklearn.linear_model import LinearRegression
model_lr = LinearRegression()
model_lr.fit(x_train_scaled, y_train)

# saving the model
with open('ModelsPrediction/Mpesa_LinearRegression.pkl', 'wb') as f:
    pickle.dump(model_lr, f)

In [76]:
y_pred = model_lr.predict(x_test_scaled)

In [77]:
# Evaluate the model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, f1_score, precision_score, confusion_matrix, roc_auc_score, recall_score, classification_report

mse = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Regression Metrics")
print()
print(f"Root mean square Error {rmse:.2f}")
print(f"Mean Absolute Error {mae:.2f}")
print(f"R2 Score: {r2:.2f}")

Regression Metrics

Root mean square Error 0.00
Mean Absolute Error 0.00
R2 Score: 1.00


In [ ]:
# visualizing the model
plt.figure(figsize=(18, 7))
plt.plot(y_test.values, label='Actual', alpha=0.7, color='#439534')
plt.plot(y_pred, label='Predicted', alpha=0.7, color='#E53631')
plt.legend()
plt.title("Actual vs Predicted Spending linear regression", fontsize=15)
plt.savefig("ModelCharts/Actual_vs_predicted_spending_linear_regression.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [79]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    random_state=42
)
rf.fit(x_train_scaled, y_train)

with open('ModelsPrediction/Mpesa_RandomForestRegression.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [80]:
y_pred_rf = rf.predict(x_test_scaled)

In [81]:
# Evaluate the  Random Forest model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest")
print()
print(f"Root Mean Square Error: {rmse_rf:.2f}") 
print(f"Mean Absolute Error: {mae_rf:.2f}")
print(f"R²: {r2_rf:.2f}")

Random Forest

Root Mean Square Error: 0.95
Mean Absolute Error: 0.15
R²: 1.00


In [ ]:
# visualizing the model
plt.figure(figsize=(18, 7))
plt.plot(y_test.values, label='Actual', alpha=0.7, color='#439534')
plt.plot(y_pred_rf, label='Predicted', alpha=0.7, color='#E53631')
plt.legend()
plt.title("Actual vs Predicted Spending using random forest", fontsize=15)
plt.savefig("ModelCharts/Actual_vs_predicted_spending_using_random_forest.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [105]:
df_encoded

,Receipt,transaction_Day,Year,Month,Date,Weekday,Hour,Minute,Seconds,Transaction_type,Transaction_party,Transaction_amount,paid_in_or_Withdraw,Balance
0,TC17XS3U67,49,2025,3,1,5,15,55,18,19,13,0.0,0,27.00
1,TC14XHM64C,49,2025,3,1,5,14,49,55,10,13,0.0,0,107.00
2,TC11XF54B3,49,2025,3,1,5,14,34,40,18,13,60.0,0,1107.00
3,TC13WSE6N5,49,2025,3,1,5,12,11,53,13,17,0.0,0,7.00
4,TBS1THWSOL,48,2025,2,28,4,17,50,41,19,13,0.0,0,17.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,SJ49WDO7A3,1,2024,10,4,4,18,38,59,4,4,50.0,1,2330.81
1483,SJ49WC912X,1,2024,10,4,4,18,31,59,4,13,170.0,1,2380.81
1484,SJ41W9ZSH5,1,2024,10,4,4,18,22,0,4,6,130.0,1,2550.81
1485,SJ40V94Q5K,1,2024,10,4,4,14,35,59,15,13,50.0,1,2680.81


In [83]:
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators =100, learning_rate = 0.1, max_depth=5,random_state=42)
xgb.fit(x_train_scaled, y_train)

# saving the model

with open('ModelsPrediction/Mpesa_XGBRegressor.pkl', 'wb') as f:
    pickle.dump(xgb, f)

In [84]:
y_pred_xgb = xgb.predict(x_test_scaled)

In [152]:
# Evaluating the xgboost model
from sklearn.metrics import (
    accuracy_score, 
    precision_score) 
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mse_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost")
print()
print(f"Root Mean Square Error: {rmse_xgb:.2f}")
print(f"Mean Absolute Error: {mae_xgb:.2f}")
print(f"R2: {r2_xgb:.2f}")

XGBoost

Root Mean Square Error: 1.28
Mean Absolute Error: 0.19
R2: 1.00


In [ ]:
# visualizing the model
plt.figure(figsize=(18, 7))
plt.plot(y_test.values, label='Actual', alpha=0.7, color='#439534')
plt.plot(y_pred_xgb, label='Predicted', alpha=0.7, color='#E53631')
plt.legend()
plt.title("Actual vs Predicted Spending using xgboost", fontsize=15)
plt.savefig("ModelCharts/Actual_vs_predicted_spending_using_xgboost.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [87]:
# making duplicate data
forecast = combined_data.copy()
forecast.rename(columns={
    "Date": "Day"
} ,inplace=True)

In [88]:
# Forecasting using Prophet
forecast["transation_time"] = pd.to_datetime(forecast[["Year", "Month", "Day", "Hour", "Minute", "Seconds"]])
forecast = forecast.sort_values("transation_time")

daily_df = forecast.groupby(forecast['transation_time'].dt.date)['Transaction_amount'].sum().reset_index()

daily_df.columns = ['ds', 'y']

In [97]:
# using prophet to model
from prophet import Prophet

model_pr = Prophet()
model_pr.fit(daily_df)

# saving the prophet model

with open('ModelsPrediction/Mpesa_prediction_prophet.pkl', 'wb') as f:
    pickle.dump(model_pr, f)

00:48:42 - cmdstanpy - INFO - Chain [1] start processing
00:48:42 - cmdstanpy - INFO - Chain [1] done processing


In [131]:
future = model_pr.make_future_dataframe(periods=60)
forecast = model_pr.predict(future)

In [ ]:
# visualizing the model
fig = model_pr.plot_components(forecast)
# fig = model.plot(forecast)
plt.title("M-pesa daily spend forecast")
plt.xlabel("Date")
plt.ylabel("Spend amount")
plt.savefig("ModelCharts/mpesa_daily_spending_forecast_using_prophet.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [93]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(daily_df_arima['y'],
                order=(1, 0, 1),               # Non-seasonal part (p, d, q)
                seasonal_order=(1, 1, 1, 12),  # Seasonal part (P, D, Q, m), m=12 for yearly seasonality
                enforce_stationarity=False,    
                enforce_invertibility=False)

# Fit the SARIMA model
model_fit = model.fit()


# Forecast for the next 60 periods
forecast_sarima = model_fit.forecast(steps=60)
joblib.dump(model_fit, "ModelsPrediction/Mpesa_prediction_SARIMA")

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            5     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.13036D+00    |proj g|=  1.46736D-01

At iterate    5    f=  9.10077D+00    |proj g|=  1.26643D-02


 This problem is unconstrained.



At iterate   10    f=  9.10036D+00    |proj g|=  2.28652D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    5     12     16      1     0     0   2.646D-05   9.100D+00
  F =   9.1003571285590699     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [ ]:
# Visualizing the ARIMA Prediction
plt.figure(figsize=(18, 7))
plt.plot(daily_df.index, daily_df['y'], label='History', color='#E53631')
plt.plot(pd.date_range(daily_df.index[-1], periods=60, freq='D'), forecast_sarima, label='Forecast', color='#439534')
plt.title('M-pesa spending forecast using SARIMA')
plt.legend()
plt.savefig("ModelCharts/mpesa_spending_forecast_using_SARIMA.png", dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [211]:
combined_data.to_csv("Datasets/Mpesa_cleaned_data.csv")

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit the ARIMA model with specified order (example: (1, 0, 1))
model = ARIMA(daily_df_arima['y'], order=(1, 0, 1))

# Fit the model
model_fit = model.fit()

# Forecast for the next 60 periods
forecast_arima = model_fit.forecast(steps=60)

In [ ]:
combined_data['Transaction_type'].unique()

In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder

# Load your original dataset
data_path = "/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/Datasets/Mpesa_cleaned_data.csv"
df = pd.read_csv(data_path)

# Define the columns to encode
categorical_columns = ['Transaction_type', 'Transaction_party', 'paid_in_or_Withdraw']

# Create and fit a LabelEncoder for each column, then save separately
for col in categorical_columns:
    encoder = LabelEncoder()
    df[col] = df[col].astype(str)  # in case any missing values or weird datatypes
    encoder.fit(df[col])

    # Save the encoder
    encoder_filename = f"/Users/briankimanzi/Documents/programming Languages/PythonProgramming/JupyterNoteBook/ModelsPrediction/{col}_encoder.pkl"
    with open(encoder_filename, 'wb') as f:
        pickle.dump(encoder, f)

    print(f"Saved encoder for '{col}' at {encoder_filename}")